In [3]:
from array import array
import os.path
BaseLayersNow = ["DailyTempRange11_20.tif","MinTemp11_20.tif","MaxTemp11_20.tif","CO_ConstantRaster.tif","PET11_20.tif","RainFall11_20.tif","AnnualTempRange11_20.tif",\
              "SeasonalityRainFall11_20.tif","SeasonalityPET11_20.tif","Isothermonality11_20.tif","DailyTemperature11_20.tif","CWD11_20.tif",\
              "GrowthSeasonLength11_20.tif","TotalSolarNormalisedforCalc.tif"]
BaseLayersFuture = ["DailyTempRange90_99.tif","MinTemp90_99.tif","MaxTemp90_99.tif","CO_ConstantRasterFuture.tif","PET90_99Recalculated.tif",\
                    "RainFall90_99Recalculated.tif","AnnualTempRange90_99.tif","SeasonalityRainFall90_99.tif","SeasonalityPET90_99.tif",\
                    "Isothermonality90_99.tif","DailyTemperature90_99.tif","CWD90_99Recalculated.tif","GrowthSeasonLength90_99.tif","TotalSolarNormalisedforCalc.tif"]

AllFields = ["PowerAx1","PowerAx2","B_1","B_2","B_3","B_4","B_5","B_6","B_7","B_8","B_9","B_10",\
             "B_11","B_12","B_13","B_14","B_Constant",\
             "Max_1","Max_2","Max_3","Max_4","Max_5","Max_6","Max_7","Max_8","Max_9",\
             "Max_10", "Max_11","Max_12","Max_13"]
GlobalMax = [0.282538182,1,0.403550081,0.231530396,0.326208017,0.846732917,0.421948087,0.745014208,0.460737477,0.398952798,1,1,0.754871795,1]
GlobalMin = [0.717461818,0,0.596449919,0.768469604,0.673791983,0.153267083,0.578051913,0.254985792,0.539262523,0.601047202,0,0,0.245128205,0]
FolderIn = "C:\DannyData\Projects\Agrichange\LayersforGlobalPLS\\"
TempFolder = "C:\DannyData\Projects\Agrichange\LayersforGlobalPLS\TemporaryFiles\\"
OutFolder = "C:\DannyData\Projects\Agrichange\LayersforGlobalPLS\PerLTEFiles\\"
DeleteFact = 1   # 1 means yes delete the factor layers after combining
Current = 1 # 1 if Current, 0 if future
RunDis = 0; # 1 to run the distances
SuitabilitySuffic = "MinimumSuitabilityInBetween.tif" 
SuitabilitySufficFuture = "MinimumSuitabilityInBetweenFuture.tif"

#FileNames = ["Ax1PLSMaxIndividual","Ax2PLSMaxIndividual"]
FileNames = ["Axis1PLSMinimumIndividual","Axis2PLSMinimumIndividual"]
if Current== 1:
    Layers = BaseLayersNow
elif Current== 0:
    Layers = BaseLayersFuture
    
for NumberLTE in range(1,20):
    # Make arrays from Coefficients
    exp = "ID = " + str(NumberLTE)
    arcpy.analysis.Select(in_features=FileNames[0],out_feature_class="SelectLTE",where_clause=exp)
    fields = ["LTE"]
    rows = arcpy.SearchCursor("SelectLTE") 
    for row in rows: 
        for field in fields:
            LTE= row.getValue(field)
    if Current== 1:
        OutFile = OutFolder + LTE + "MinimumSuitability.tif" # OUTFILE
    elif Current== 0:
        OutFile = OutFolder + LTE + "MinimumSuitabilityFuture.tif" # OUTFILE
    EndMessage = " for LTE: # " + str(NumberLTE) +" being " + LTE + "                 "
    fields = ["Area"]
    rows = arcpy.SearchCursor("SelectLTE") 
    for row in rows: 
        for field in fields:
            Area= row.getValue(field)
    
    CoefficientsAX1 = array('f')
    for FieldAll in range(0,len(AllFields)):
        fields = [AllFields[FieldAll]]
        rows = arcpy.SearchCursor("SelectLTE") 
        for row in rows: 
            for field in fields:
                rVal= row.getValue(field)
                CoefficientsAX1.append(rVal)
    arcpy.analysis.Select(in_features=FileNames[1],out_feature_class="SelectLTE2",where_clause=exp)  
    CoefficientsAX2 = array('f')
    for FieldAll in range(0,len(AllFields)):
        fields = [AllFields[FieldAll]]
        rows = arcpy.SearchCursor("SelectLTE2") 
        for row in rows: 
            for field in fields:
                rVal= row.getValue(field)
                CoefficientsAX2.append(rVal)
    # Generate the per LTE layers
    Count = 0;
    for Lay in range(0,len(Layers)):
        Message = "Doing Layer: #" + str(Lay) + EndMessage                 
        print(Message, end="\r") 
        MaxCount = Lay + 17 # -1 for cut-off first field
        BetaCount = Lay + 2 # -1 for cut-off first field
        FactorName1 = "Factor_" + str(1) + "_" + str(Lay)
        FactorName2 = "Factor_" + str(2) + "_" + str(Lay)
        LayerDone = FolderIn + Layers[Lay]
        LayerNameAfterDone = TempFolder + Layers[Lay] + "_Norm" + ".tif"
        if Lay != 13:
            DivideTemp = arcpy.sa.Divide(in_raster_or_constant1= LayerDone,in_raster_or_constant2=CoefficientsAX1[MaxCount])
            MinusTemp = arcpy.sa.Minus(in_raster_or_constant1=DivideTemp,in_raster_or_constant2=GlobalMin[Lay])
            out_raster = arcpy.sa.Divide(in_raster_or_constant1= MinusTemp,in_raster_or_constant2=GlobalMax[Lay])  
            out_raster.save(LayerNameAfterDone)
        elif Lay == 13:
            arcpy.management.CopyRaster(in_raster=LayerDone,out_rasterdataset=LayerNameAfterDone,
                config_keyword="",background_value=None,nodata_value="-999",onebit_to_eightbit="NONE",
                colormap_to_RGB="NONE",pixel_type="",scale_pixel_value="NONE",RGB_to_Colormap="NONE",
                format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",build_multidimensional_transpose="NO_TRANSPOSE")
        
    # Multiply layers with coefficients
        if Count == 0:
            LayersArrayFull_1= FactorName1
            LayersArrayFull_2= FactorName2
            LayersNameAfterDone = LayerNameAfterDone;
        else:
            LayersArrayFull_1 =  LayersArrayFull_1 + ";" + FactorName1
            LayersArrayFull_2 =  LayersArrayFull_2 + ";" + FactorName2
            LayersNameAfterDone =LayersNameAfterDone + ";" + LayerNameAfterDone
        Count = Count + 1
        out_raster = arcpy.sa.Times(in_raster_or_constant1=CoefficientsAX1[BetaCount],in_raster_or_constant2=LayerNameAfterDone)
        out_raster.save(FactorName1)
        out_raster = arcpy.sa.Times(in_raster_or_constant1=CoefficientsAX2[BetaCount],in_raster_or_constant2=LayerNameAfterDone)
        out_raster.save(FactorName2)   
    Message = "Adding Layers together" + EndMessage
    print(Message, end="\r")     
    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull_1,statistics_type="SUM",
                    ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",percentile_value=90, percentile_interpolation_type="AUTO_DETECT") 
    out_raster.save("PLSAxis_1Tmp")
    out_raster = arcpy.sa.Plus(in_raster_or_constant1="PLSAxis_1Tmp",in_raster_or_constant2=CoefficientsAX1[16])#-1 for cut-off first field
    out_raster.save("PLSAxis_1")
    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull_2,statistics_type="SUM",
                    ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",percentile_value=90, percentile_interpolation_type="AUTO_DETECT")
    out_raster.save("PLSAxis_2Tmp")
    out_raster = arcpy.sa.Plus(in_raster_or_constant1="PLSAxis_2Tmp",in_raster_or_constant2=CoefficientsAX2[16])#-1 for cut-off first field
    out_raster.save("PLSAxis_2")
    Message = "Cleaning Layers from Memory" +" for LTE: #" + LTE + "                   "
    print(Message, end="\r")   
    if DeleteFact != 0:
        arcpy.Delete_management(LayersArrayFull_1)
        arcpy.Delete_management(LayersArrayFull_2)
        arcpy.Delete_management( LayersNameAfterDone)
    Message = "Including axis weights" + EndMessage
    print(Message, end="\r")
    # Redefine Axis weights as proportion
    PLSAxisSum = CoefficientsAX1[0] + CoefficientsAX1[1] #-1 for cut-off first field                                   
    PLSAxis1_Weight = CoefficientsAX1[0]/PLSAxisSum
    PLSAxis2_Weight = CoefficientsAX1[1]/PLSAxisSum      
    out_raster = arcpy.sa.Times(in_raster_or_constant1= PLSAxis1_Weight,
                       in_raster_or_constant2="PLSAxis_1")
    out_raster.save("PLSTimes1")
    out_raster = arcpy.sa.Times(in_raster_or_constant1= PLSAxis2_Weight,
                       in_raster_or_constant2="PLSAxis_2")
    out_raster.save("PLSTimes2")
    Message = "Combine axis" + EndMessage
    print(Message, end="\r")
    with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
        out_raster= arcpy.sa.Plus(in_raster_or_constant1= "PLSTimes1",in_raster_or_constant2="PLSTimes2")
        out_raster.save("CombiLayers")
        arcpy.management.Clip("CombiLayers","","ClipFile" , Area, "-999", "ClippingGeometry", "NO_MAINTAIN_EXTENT")
    out_raster = arcpy.sa.Con(in_conditional_raster="ClipFile",in_true_raster_or_constant=0,
        in_false_raster_or_constant="ClipFile",where_clause="VALUE <= 0")
    out_raster.save("ConFile")  
    out_raster.save(OutFile)  
    
    # For Current Only
    Message = "Generating Con-ranges" + EndMessage
    print(Message, end="\r")
    if Current == 1:
        OutFileInBetween = OutFolder + LTE + SuitabilitySuffic # OUTFILE IN BETWEEN
        out_raster = arcpy.sa.SetNull(in_conditional_raster="ConFile",
            in_false_raster_or_constant="ConFile",where_clause="VALUE > 1.5")
        out_raster.save("ConFile2")  
        out_raster = arcpy.sa.SetNull(in_conditional_raster="ConFile2",
            in_false_raster_or_constant="ConFile2",where_clause="VALUE < 0.5")
    elif Current == 0:
        OutFileInBetween = OutFolder + LTE + SuitabilitySufficFuture # OUTFILE IN BETWEEN FUTURE
        OutFileCurrent = OutFolder + LTE + SuitabilitySuffic # OUTFILE IN BETWEEN
        out_raster = arcpy.sa.IsNull(in_raster= OutFileCurrent)
        out_raster.save("IsNullRast")
        out_raster = arcpy.sa.SetNull(in_conditional_raster="IsNullRast",
            in_false_raster_or_constant="ConFile",where_clause="Value = 1")
    out_raster.save(OutFileInBetween)
        
    # Distance Raster for Area
    if RunDis == 1:
        if Current == 1:
            Message = "Distance raster" + EndMessage
            print(Message, end="\r")
            DistanceOutFile = OutFolder + LTE + "Distance.tif"
            with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
                      cellSize=0.01, extent = "ConFile"):
                out_raster = arcpy.sa.DistanceAccumulation(in_source_data="SelectLTE",
                    in_barrier_data=None,in_surface_raster="PLSTimes1",in_cost_raster=None,
                    in_vertical_raster=None,vertical_factor="BINARY 1 -30 30",in_horizontal_raster=None,
                    horizontal_factor="BINARY 1 45",out_back_direction_raster=None,out_source_direction_raster=None,
                    out_source_location_raster=None,source_initial_accumulation=None,
                    source_maximum_accumulation=None,source_cost_multiplier=None,source_direction="",distance_method="PLANAR")
                out_raster.save("TempDistance")
                arcpy.management.Clip("TempDistance","", "DisOutPut", Area, "-999", "ClippingGeometry", "NO_MAINTAIN_EXTENT")
                out_raster = arcpy.sa.IsNull(in_raster= OutFileInBetween)
                out_raster.save("IsNullRast")
                out_raster = arcpy.sa.SetNull(in_conditional_raster="IsNullRast",
                in_false_raster_or_constant="DisOutPut",where_clause="Value = 1")
                out_raster.save(DistanceOutFile)
    ExtraLayersToRemove = ["SelectLTE","SelectLTE2","PLSAxis_1Tmp","PLSAxis_2Tmp","PLSAxis_1","PLSAxis_2","PLSTimes1","PLSTimes2",\
                           "ConFile","ConFile2","DisOutPut","IsNullRast","TempDistance","ClipFile",DivideTemp,MinusTemp]
    arcpy.Delete_management(ExtraLayersToRemove)
    sys.stdout.flush()
Message = "Finished                                           "
print(Message)

Finished                                                                           


In [6]:
arcpy.analysis.Select(in_features="Axis1PLSMinimumIndividual",out_feature_class="SelectLTEsTemp",where_clause="Area = 'US&Mexico'")
arcpy.analysis.Select(in_features="SelectLTEsTemp",out_feature_class="SelectLTEs",where_clause="CarbonCycl = 'C4'")

OutFolder = "C:\DannyData\Projects\Agrichange\LayersforGlobalPLS\PerLTEFiles\\"
DistanceSumFile = OutFolder + "US&MexicoC4SumDistances.tif"
RelativeDistanceSumFile = OutFolder + "US&MexicoC4RelativesumDistances.tif"
FinalOutFile =    OutFolder + "US&MexicoC4WeightedSuitabilityMinimumFuture.tif" # change to Future
SuitabilitySuffic = "MinimumSuitabilityInBetweenFuture.tif" # change to Future
Nums = 26 # Europe 67 (C4: 13; C3: 54); SubSaharanAfrica: 42; Asia 13; Australia 2; Brazil: 6; US&Mexico: 50 (C4: 26; C3: 25 )
Done = 1
if Done != 1:
    Count = 0
    for NumberLTE in range(1,Nums):
        exp = "OBJECTID = " + str(NumberLTE)
        arcpy.analysis.Select(in_features="SelectLTEs",out_feature_class="SelectLTE",where_clause=exp)
        fields = ["LTE"]
        rows = arcpy.SearchCursor("SelectLTE") 
        for row in rows: 
            for field in fields:
                LTE= row.getValue(field)
        DistanceOutFile = OutFolder + LTE + "Distance.tif"                   
        if Count == 0:
            LayersArrayFull= DistanceOutFile
        else:
            LayersArrayFull =  LayersArrayFull + ";" + DistanceOutFile
        Count = Count + 1
    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,statistics_type="SUM",
                 ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",percentile_value=90, percentile_interpolation_type="AUTO_DETECT")
    out_raster.save("DistanceSum")
    out_raster.save(DistanceSumFile)

    Message = ("First phase Done")
    print(Message, end="\r")

    # Per LTE
    CountRel = 0
    for NumberLTE in range(1,Nums):
        exp = "OBJECTID = " + str(NumberLTE)
        arcpy.analysis.Select(in_features="SelectLTEs",out_feature_class="SelectLTE",where_clause=exp)
        fields = ["LTE"]
        rows = arcpy.SearchCursor("SelectLTE") 
        for row in rows: 
            for field in fields:
                LTE= row.getValue(field)
        Message = "Distance Calculation for LTE: #" + LTE + "                   "
        print(Message, end="\r")  
        DistanceOutFile = OutFolder + LTE + "Distance.tif" 
        DistanceRelativeFileTmp = OutFolder + "RelativeDistancesCURRENT\\" + LTE + "fromDistance.tif" 
        out_raster = arcpy.sa.Plus(in_raster_or_constant1=DistanceOutFile,in_raster_or_constant2=0.01)
        out_raster.save("PlusFile")
        Maxi = arcpy.sa.Raster("PlusFile").maximum
        Clause = "VALUE >= " + str(Maxi)
        out_raster = arcpy.sa.Con(in_conditional_raster="PlusFile",in_true_raster_or_constant=Maxi,
            in_false_raster_or_constant="PlusFile",where_clause= Clause)
        out_raster.save("ConFile")                
        DivideRast = arcpy.sa.Divide(in_raster_or_constant1= "ConFile",in_raster_or_constant2="DistanceSum")
        out_raster = arcpy.sa.Divide(in_raster_or_constant1= 1,in_raster_or_constant2=DivideRast)
        out_raster.save(DistanceRelativeFileTmp)
        if CountRel == 0:
            LayersArrayFullRel= DistanceRelativeFileTmp
        else:
            LayersArrayFullRel =  LayersArrayFullRel + ";" + DistanceRelativeFileTmp                
        CountRel = CountRel + 1                    

    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRel,statistics_type="SUM",
            ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
    out_raster.save("RelativeSUM")
    out_raster.save(RelativeDistanceSumFile)
    Message = ("Second phase Done                             ")
    print(Message, end="\r")

CountLTE = 0

# Per layer weights
arcpy.analysis.Statistics(in_table="SelectLTEs",out_table="Stats",statistics_fields="YearsRun MEAN",case_field="Ones",concatenation_separator="")
fields = ["MEAN_YearsRun"]
rows = arcpy.SearchCursor("Stats") 
for row in rows: 
     for field in fields:
        Mean= row.getValue(field)
arcpy.management.AddField("SelectLTEs", "WeightRatio", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
expres = "!YearsRun!/"+ str(Mean)
arcpy.management.CalculateField(in_table="SelectLTEs",field="WeightRatio",expression=expres, expression_type="PYTHON3",code_block="",field_type="TEXT",enforce_domains="NO_ENFORCE_DOMAINS")


for NumberLTE in range(1,Nums):
    exp = "OBJECTID = " + str(NumberLTE)
    arcpy.analysis.Select(in_features="SelectLTEs",out_feature_class="SelectLTE",where_clause=exp)
    fields = ["LTE"]
    rows = arcpy.SearchCursor("SelectLTE") 
    for row in rows: 
        for field in fields:
            LTE= row.getValue(field)
    Message = "Weighting for LTE: #" + LTE + "                   "
    print(Message, end="\r")   
    DistanceRelativeFileTmp = OutFolder + "RelativeDistancesCURRENT\\" + LTE + "fromDistance.tif" 
    DistanceRelativeFile = OutFolder + "RelativeDistancesCURRENT\\" + LTE + "WeightedDistance.tif" 
    out_raster = arcpy.sa.Divide(in_raster_or_constant1=DistanceRelativeFileTmp,in_raster_or_constant2=RelativeDistanceSumFile)
    out_raster.save("CorrectedRelativePerLTE")
    out_raster.save(DistanceRelativeFile)
    #arcpy.Delete_management(DistanceRelativeFileTmp)
    # Suitability multiplication
    OutFileInBetween = OutFolder + LTE + SuitabilitySuffic # OUTFILE IN BETWEEN     
    InBetweenName = "InBetween_" + LTE
    with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
              cellSize=DistanceRelativeFile):
        out_raster = arcpy.sa.Times(in_raster_or_constant1=OutFileInBetween,
                       in_raster_or_constant2="CorrectedRelativePerLTE") 
        out_raster.save("TimesRast")
        fields = ["WeightRatio"]
        rows = arcpy.SearchCursor("SelectLTE") 
        for row in rows: 
            for field in fields:
                Weight= row.getValue(field)
        out_raster = arcpy.sa.Times(in_raster_or_constant1="TimesRast",
                       in_raster_or_constant2=Weight)         
        out_raster.save(InBetweenName)         
    if CountLTE == 0:
        LayersArrayFullCombi=InBetweenName
    else:
        LayersArrayFullCombi =  LayersArrayFullCombi + ";" + InBetweenName                
    CountLTE = CountLTE + 1 
Message = ("Combining all weighted layers                ")
print(Message)
out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullCombi,statistics_type="SUM",
            ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
out_raster.save("CombinedWeightedSuitability")
out_raster.save(FinalOutFile)
Message = ("Finished                              ")
print(Message)

Combining all weighted layers                     
Finished                              
